In [2]:
using SeisNoise, SeisIO, Plots, HDF5
using Distributed # for parallel computing

# Set station and channel dynamically
station1 = "HYS14"
station2 = "HYSB1"
channel1 = "BHZ"
channel2 = "BHZ"

fs = 40.0 # sampling frequency in Hz
freqmin,freqmax = 0.01,16.0 # min and max frequencies
cc_step, cc_len = 450, 1800 # corrleation step and length in S
maxlag = 200. # maximum lag time in correlation

# Load StationXML metadata once at the start
resp_file_1 = "/data/wsd02/maleen_data/OOI-Data/StationXML/OO.$(station1)..$(channel1).xml" 
resp_file_2 = "/data/wsd02/maleen_data/OOI-Data/StationXML/OO.$(station2)..$(channel2).xml" 
meta1 = SeisIO.read_meta("sxml",resp_file_1)
meta2 = SeisIO.read_meta("sxml",resp_file_2)

# Start a parallel computing pool
num_cores = Sys.CPU_THREADS
#addprocs(10) # use all available cores

for j in 2014:2023
    for i in 1:366
        # We have used string interpolation here to automatically insert station and channel values
        a = "/data/wsd02/maleen_data/OOI-Data/$(station1)/$(j)/$(lpad(i, 3, '0'))/$(station1).OO.$(j).$(lpad(i, 3, '0')).$(channel1)"
        b = "/data/wsd02/maleen_data/OOI-Data/$(station2)/$(j)/$(lpad(i, 3, '0'))/$(station2).OO.$(j).$(lpad(i, 3, '0')).$(channel2)"

        try
            S1 = SeisIO.read_data(a)
            S2 = SeisIO.read_data(b)
            S1.fs = [40.0]
            S2.fs = [40.0]

            SeisIO.translate_resp!(S1, meta1.resp[1], chans=1)
            SeisIO.translate_resp!(S2, meta1.resp[1], chans=1)

            SeisIO.remove_resp!(S1)
            SeisIO.remove_resp!(S2)

            process_raw!(S1, fs)
            process_raw!(S2, fs)

            R = RawData.([S1,S2], cc_len, cc_step)
            detrend!.(R)
            taper!.(R)
            bandpass!.(R, freqmin, freqmax, zerophase=true)
            onebit!.(R)
            FFT = rfft.(R)
            whiten!.(FFT, freqmin, freqmax)
            C = correlate(FFT[1], FFT[2], maxlag)
            save_corr(C, "/data/wsd02/maleen_data/Correl_seismic/")

        catch
            println("cannot process " * a)
        end
    end
end